In [1]:
seed = 1
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
import torch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import os
import sklearn.cluster as cluster
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

from sklearn.metrics.pairwise import euclidean_distances
import joblib
import subprocess

/home/laks/36env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/laks/36env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/laks/36env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/laks/36env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarnin

## Данные

Подробнее о данных см. ноутбук ELMo

In [2]:
data_path = 'data'
test = pd.read_csv(os.path.join(data_path, 'main/wiki-wiki/test-solution.csv'), sep='\t')
test.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,440,банка,1,NaN,"7-12, 36-41",здание банка китая ( макао ) здание банка кита...
1,441,банка,2,NaN,"13-18, 201-206",трехлитровая банка во времена ссср такие банки...
2,442,банка,1,NaN,"21-26, 58-63, 101-106, 263-268, 538-543",здание национального банка украины в здании на...
3,443,банка,1,NaN,"134-139, 262-267, 345-350",зюдфиртель города эссен ( федеральная земля се...
4,444,банка,1,NaN,"43-48, 300-305, 381-386, 521-526",ведения боевых действий полевые учреждения бан...


## Модель

In [3]:
model = SentenceTransformer('distiluse-base-multilingual-cased')
model.device

device(type='cuda')

In [4]:
corpus_embeddings = model.encode(test['context'].to_list())
if not os.path.exists('vecs'):
    os.mkdir('vecs')
joblib.dump(corpus_embeddings, 'vecs/sent-tr-test.joblib')

['vecs/sent-tr-test.joblib']

При обучении sentence-transformers использовались метрики близости векторов, поэтому для кластеризации лучше работает не K-средних, а SpectralClustering на матрице расстояний.

## Результаты на test

In [5]:
res=[]
test['vecs']=joblib.load('vecs/sent-tr-test.joblib')
groups= test.groupby('word')
for group in groups:
    group=group[1]
    vectors=group['vecs'].tolist()
    sim_matrix = euclidean_distances(vectors, vectors)
    labels = cluster.SpectralClustering(n_clusters=len(group['gold_sense_id'].unique()),n_init=12,gamma=0.5,
                                        random_state=seed).fit_predict(sim_matrix)
    group.predict_sense_id = labels
    res.append(group)
out = pd.concat(res)
out.to_csv('outs/sent-tr-test.csv', sep='\t')

command = f"../../36env/bin/python evaluate.py 'outs/sent-tr-test.csv'; exit 0"
print(subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT, ).decode('utf-8'))

word	ari	count
банка	0.265580	112
белка	0.716129	135
бит	1.000000	84
горе	0.580798	60
граф	0.972907	148
душ	1.000000	79
	0.757703	618

